In [ ]:
#Importaciones necesarias para que funcione el codigo
import pandas as pd
import pyreadr
import os
import numpy as np
import ast
import json

Aqui abajo vamos a crear la carpeta "insumos" que es donde guardaremos los insumos necesarios, tambien en este caso se almacenan las salidas.

In [ ]:
# Nombre de la carpeta donde antes de ejecutar el script se guardan los datos
carpeta = "insumos"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

Este codigo de abajo lee la base de datos y le da el nombre de df(tipo de Dato DataFrame).

In [ ]:
df = pd.read_csv("insumos/reportTQ.csv", delimiter=";")
#Leer el archivo de esa ruta

Con este codigo vemos la dimension de la df.

In [ ]:
df.shape

El .head nos ayuda a ver un resumen de la base de datos.

In [ ]:
df.head()

Estos codigos son metodos que serviran mas adelante para buscar todas las columnas en formato lista y todas las columnas en formato json.


!! Recuerda que solo se ejecuta una funcion una vez se llame, aqui solo las estamos definiendo (No hacen nada aun).

In [ ]:
def find_list_columns(df):
    list_columns = []
    for col in df.columns:
        try:
            if df[col].apply(lambda x: isinstance(x, list) or isinstance(ast.literal_eval(x), list) if isinstance(x, str) else False).any():
                list_columns.append(col)
        except (ValueError, SyntaxError):
            continue  # Ignora errores de conversión
    return list_columns

def find_json_columns(df):
    json_columns = []
    for col in df.columns:
        try:
            if df[col].apply(lambda x: isinstance(x, dict) or isinstance(json.loads(x), dict) if isinstance(x, str) else False).any():
                json_columns.append(col)
        except (json.JSONDecodeError, TypeError):
            continue
    return json_columns

Expande una columna que contiene listas en múltiples columnas, 
    mostrando el valor si está presente y NaN si no.

Y el otro Expande columnas específicas que contienen JSON en múltiples columnas con claves como nombres.

In [ ]:
def expand_list_column_text(df, column_name):
    """
    Expande una columna que contiene listas en múltiples columnas, 
    mostrando el valor si está presente y NaN si no.

    Parámetros:
    - df: DataFrame de pandas.
    - column_name: Nombre de la columna a expandir.

    Retorna:
    - DataFrame modificado con nuevas columnas de texto.
    """
    unique_values = set()

    # Paso 1: Extraer todos los valores únicos sin repetir
    for row in df[column_name].dropna():
        try:
            parsed_list = json.loads(row) if isinstance(row, str) else row
            if isinstance(parsed_list, list):
                unique_values.update(parsed_list)  # Agregar valores únicos
        except json.JSONDecodeError:
            continue  # Ignorar errores en JSON mal formateado

    # Paso 2: Crear nuevas columnas con el valor en texto si está presente
    i=1
    for value in sorted(unique_values):  # Ordenamos para mantener consistencia
        col_name = f"{column_name}_opcion{i}_{value}"
        df[col_name] = df[column_name].apply(
            lambda x: value if isinstance(x, str) and value in json.loads(x) else None
        )
        i+=1
    return df

def expand_json_columns(df, json_col):
    """
    Expande columnas específicas que contienen JSON en múltiples columnas con claves como nombres.
    
    Parámetros:
    - df: DataFrame de pandas con algunas columnas que contienen JSON.
    - json_cols: Lista de nombres de columnas a expandir.
    
    Retorna:
    - DataFrame con las columnas JSON expandidas.
    """
    new_columns = {}
    column=json_col
    if column in df.columns:
        sample_data = df[column].dropna().astype(str).values
        
        for entry in sample_data:
            try:
                parsed_json = json.loads(entry)
                if isinstance(parsed_json, dict):
                    for key in parsed_json.keys():
                        new_col_name = f"{column}_{key}"
                        if new_col_name not in new_columns:
                            new_columns[new_col_name] = []
            except json.JSONDecodeError:
                continue
    
    # Crear nuevas columnas en el DataFrame
    i=1
    for new_col in new_columns.keys():
        original_col, key = new_col.rsplit('_', 1)
        new_col_name = f"{original_col}_opcion{i}_{key}"  # Agregar "opcion" en el nombre
        df[new_col_name] = df[original_col].apply(lambda x: json.loads(x).get(key) if isinstance(x, str) else None)
        i+=1
    
    return df

## Ejecuta esto si quieres modificar manualmente la lista de JSON Colums y la lista de List Colums

In [ ]:
json_cols = [
    "826 - La empresa",
    "827 - Fondo de empleados de la empresa",
    "828 - Entidad financiera y/o tarjetas de crédito",
    "829 - Prestamista/Gota a gota",
    "830 - Icetex",
    "831 - Familiar o amigo",
    "832 - Comfandi/Caja de Compensación Familiar",
    "833 - Otro",
    "834 - Seleccione el nivel de formación de su mayor interés:",
    "851 - ¿Cuántos días realizó actividades físicas intensas? (ej. Levantar pesos pesados, correr, andar rápido o en bicicleta)"
]

In [ ]:
list_cols = [
    "899 - ¿Cuál considera que debe ser el rol de los padres en la educación de los hijos? (Seleccione las tres opciones que mejor describan su opinión)",
    "911 - ¿Cómo puede influir una buena educación en las oportunidades futuras de los hijos? (Seleccione las tres opciones que mejor describan su opinión)"
]

## Ejecuta esto si quieres que la busqueda sea automatica

In [ ]:
list_cols = find_list_columns(df)

print("Columnas con listas:", list_cols)
print("Numero de listas:", len(list_cols))

In [ ]:
json_cols = find_json_columns(df)

print("Columnas con JSON:", json_cols)
print("Numero de JSON:", len(json_cols))

In [ ]:
numJsonCols=len(json_cols)
numListCols=len(list_cols)

print("numero de columnas Json:", numJsonCols)
print("numero de columnas en List:", numListCols)

print("Numero de columnas:", df.shape[1])

## Flujo normal

Es necesario tener el "nameEspecialColumn" y el "ignore_cols" asi sea lo mismo, ya que ignore_cols se refiere a las columnas por ignorar, pueden ser una o varias, la especial si es solo una y solo funciona para esa forma en la que esta esa columna.

In [ ]:
#Nombre de la columna especial
nameEspecialColumn="785 - Describa los miembros de su familia con los que convive actualmente (Incluyéndose usted):"

# Especificar columnas a ignorar
ignore_cols = ["785 - Describa los miembros de su familia con los que convive actualmente (Incluyéndose usted):"]

In [ ]:
# Expandir columnas de listas
for list_col in list_cols:
    if list_col in ignore_cols:
            continue  # Ignorar columnas especificadas
    else:
        df = expand_list_column_text(df, list_col)


# Expandir columnas de JSON
for json_col in json_cols:
    if json_col in ignore_cols:
            continue  # Ignorar columnas especificadas
    else:
        df = expand_json_columns(df, json_col)

In [ ]:
# Filtrar list_cols para excluir las columnas en ignore_cols
cols_to_drop = [col for col in list_cols if col not in ignore_cols]
# Eliminar las columnas del DataFrame
df = df.drop(columns=cols_to_drop, errors="ignore")

# Filtrar list_cols para excluir las columnas en ignore_cols
cols_to_drop = [col for col in json_cols if col not in ignore_cols]
# Eliminar las columnas del DataFrame
df = df.drop(columns=cols_to_drop, errors="ignore")

In [ ]:
df.shape

In [ ]:
print(df)

In [ ]:

# Asegurar que no haya valores NaN en la columna antes de convertir
df[nameEspecialColumn] = df[nameEspecialColumn].fillna("[]")  # Reemplaza NaN con una lista vacía
# Convertir la columna de strings en listas de diccionarios (JSONs)
df[nameEspecialColumn] = df[nameEspecialColumn].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)  

# Encontrar la cantidad máxima de JSONs en cualquier fila
max_jsons = df[nameEspecialColumn].apply(len).max()

# Expandir los JSONs en columnas separadas
df_expanded = df[nameEspecialColumn].apply(lambda x: x + [{}] * (max_jsons - len(x)))  # Asegurar que todas las listas tengan la misma longitud
df_expanded = pd.DataFrame(df_expanded.tolist()).add_prefix(f"{nameEspecialColumn}_opcion")  # Convertir a DataFrame y agregar prefijo
df_expanded.columns = [f"{nameEspecialColumn}_opcion{i+1}" for i in range(df_expanded.shape[1])]


# Unir con el DataFrame original sin la columna inicial
df = pd.concat([df.drop(columns=[nameEspecialColumn]), df_expanded], axis=1)



# Mostrar el resultado
print(df)


In [ ]:
df.head()

In [ ]:
# Expandir cada columna de JSON en sus propias columnas
df = pd.concat(
    [df.drop(columns=df.filter(like=f"{nameEspecialColumn}_opcion").columns)] + 
    [df[f"{nameEspecialColumn}_opcion{i+1}"].apply(pd.Series).add_prefix(f"{nameEspecialColumn}_opcion{i+1}_") for i in range(max_jsons)],
    axis=1
)

# Mostrar el resultado
print(df)


In [ ]:
df.head()

Este bloque de codigo, Ordena

In [ ]:
# Lista de columnas iniciales que no deben cambiar su posición
fixed_columns = ["ID", "Survey", "TimeStart", "TimeEnd", "TimeDiff", 
                 "IsFinished", "Interviewer", "Username", "Nit", "Institution"]

# Extraer las demás columnas y ordenarlas
other_columns = sorted([col for col in df.columns if col not in fixed_columns])

# Reorganizar el DataFrame
df = df[fixed_columns + other_columns]

# Mostrar el DataFrame reorganizado
print(df.head())

In [ ]:
# Exportar a Excel
df.to_excel("insumos/reportTQ_limpiado.xlsx", index=False)

# Exportar a CSV con separador ';'
df.to_csv("insumos/reportTQ_limpiado.csv", index=False, sep=';', encoding='utf-8')

print("Archivos Excel y CSV generados correctamente.")
